# Retraitement des données

## A) Sélection des variables pertinentes pour l'étude

In [1]:
# Dans le dataframe dpe_f:
dpe_f=dpe_f.loc[:,["id","date_etablissement_dpe","consommation_energie","classe_consommation_energie","estimation_ges","classe_estimation_ges","annee_construction","surface_thermique_lot","commune","code_insee_commune_actualise","longitude","latitude","tr002_type_batiment_description","tv016_departement_code","_geopoint","geometry"]]
dpe_f.head()

NameError: name 'dpe_f' is not defined

In [ ]:
# Dans le dataframe Filosofi:
Filosofi=Filosofi.loc[:,["","","","",""]]
Filosofi.head()

## B) Nettoyage et mise en forme de certaines variables

In [ ]:
# Nettoyage de la variable "code_insee_commune" dans le df "dpe_f":
# Nettoyage de la variable "code_insee_commune":
# liste des codes commune exisistant initialement dans le fichier DPE brut:
a=DPE_IDF_clean2["code_insee_commune"].unique().tolist()
# on transforme les éléments de la liste en str:
b=[str(x) for x in a]
# on code le filtre à appliquer à notre df DPE_IDF_clean3:
filtre = [x for x in b if len(x) == 5 if "." not in x if x[:2] in (["75","77","91","92","93","94","95"])]
print(len(filtre)) # nombre de codes commune retenus in fine pour l'IDF (sur la période 2014-2020).
# Filtrage de DPE_IDF_clean3:
DPE_IDF_clean3=DPE_IDF_clean2[DPE_IDF_clean3["code_insee_commune"].isin(filtre)]

In [ ]:
# Modification des types des variables: passage au format numérique de toutes les variables sauf codgeo et libgeo.
liste_var=Filosofi.columns.tolist()[2:]
for var in liste_var:
    Filosofi[var]=pd.to_numeric(Filosofi[var])
Filosofi.info()

In [ ]:
# Création d'une variable "département":
Filosofi["DEP"]=Filosofi["CODGEO"].str[:2]
# Filtrage géographique: on ne garde que les départements d'Ile-de-France:
Filosofi_IDF=Filosofi[Filosofi["DEP"].isin(["75","91","92","93","94","95","77","78"])]

## C) Création de nouvelles variables 

In [ ]:
# Calcul de nouvelles variables dans dpe_f: année du DPE, âge du logement, département.

# 1) Création d'une variable "année" de réalisation du DPE:
dpe_f["annee"]=dpe_f.date_visite_diagnostiqueur.str[:4]

# 2) Création d'une variable "âge" du logement:
# il faut d'abord transformer la variable "année" en int64:

dpe_f["age_logement"]=dpe_f["annee"]-dpe_f["annee_construction"]

# 3) Création d'une variable "département" à partir du code de la commune du DPE:
dpe_f["dep"]=dpe_f.code_insee_commune.str[:2]

## D) Agrégation du df "dpe_f" par commune: création du df "df_dpe_com" 

In [ ]:
# On commence par transformer les variables catégorielles en indicatrices:
Classe_conso_IDF=pd.get_dummies(DPE_IDF_clean3["classe_consommation_energie"])

## E) Jointure avec le dataframe "Filisofi" par commune: création du df "df_dpe_filo_com"